# PRÉ-PROCESSAMENTO
Este arquivo dita o pré-processamento dos eletrodos. A idéia básica é:

 - recuperar o nome de cada sub-árquivo que está dentro da pasta `dataset/`.
 
 - abrir os arquivos e somar os valores de cada eletrodo em todos os arquivos.

 - normalizar os dados dividindo-os pela quantidade de vez que o eletrodo apareceu nos arquivos.

 - Porfím, salvar os dados em `.json` para consulta com mais eficiência e também em `.csv`, para utilizar no restante do projeto.

# Listando todos os arquivos nos sub-diretórios

Para calcular a média de cada eletrodo em cada arquivo, é necessário recuperar o nome de cada arquivos nas sub-pastas.



In [1]:
import os

def get_sub_files(path_name, files=[]):

    subpaths = os.listdir(path_name)

    for path in subpaths:
        full_name = path_name + path
        if os.path.isdir(full_name):
            get_sub_files(full_name + '/', files) # vai para a proxima subpasta
        # tudo que for arquivo e não começar com '.' ou não for um README
        # então salva o caminho deste arquivo
        elif not path.startswith('.') and not path.__eq__('README') and not path.__contains__('.json'):
            files.append(full_name)

    return files

all_files = get_sub_files('dataset/')

# Carregando os dados dos eletrodos

Com todos os arquivos listados, basta abrir cada um e salvar os valores de cada eletrodo. Ou seja, será somado todos os eletrodos de todos os arquivos. Assim, se o eletrodo `B7`, por exemplo, estiver em mais de um arquivo, os seus valores serão somados.

In [2]:
# soma a quantidade de vezes que um eletrodo aparece nos arquivos
def electrodeSum(electrode):
    global electrodeQtd

    try:
        electrodeQtd[electrode] += 1
    except KeyError:
        electrodeQtd[electrode] = 0
        electrodeSum(electrode)

# apenas soma o valor do eletrodo com os demais, por exemplo,
# todos os eletrodos 'B7' de todos os arquivos serão somados
def insert(output, electrode, pos, value):
    try:
        output[electrode][pos] += value
        # só incrementa a quantidade quando terminar de recuperar todos os 255
        if pos == 255:
            electrodeSum(electrode)
    except KeyError:
        output[electrode] = [0]*256

        insert(output, electrode, pos, value)

# no geral, processa um determinado arquivo
def process_file(output, file):

    # para cada linha do arquivo
    for line in file.readlines():
        if line.startswith('#'):  #ignora as linhas iniciadas com #
            continue

        # 41 FP2 168 -4.720
        line = line.split()

        # recupera os valores da linha
        electrode, pos, value = line[1:4]
        insert(output, electrode, int(pos), float(value))

def process_files(files, path, output={}):

    # para todos os arquivos
    for file in files:
        if file.__contains__(path):
            try:
                process_file(output, open(file))
            except:
                print(file)

    return output

# Fazendo a média

Agora resta dividir cada um dos valores somados pela quantidade de vezes que esse eletrodo apareceu nos arquivos

In [3]:
# para cada um dos 256 valores do eletrodo,
# divide-o pela quantidade de vez que ele apareceu
def divide(electrodeList, qtd):
    for i in range(0, len(electrodeList)):
        electrodeList[i] /= qtd

def media(electrodeQtd, output):
    for electrode in list(electrodeQtd.keys()):
        divide(output[electrode], electrodeQtd[electrode])

# O problema estava no fato do Jupyter não encerrar a execução
# então cada vez que executava novamente, o Jupyter utiliza os valores que já estavam na memória

# Agora para os arquivos 'A' está funcionando
# Não sei por que não está para os arquivos 'B'
electrodeQtd = {}
outputA = {}
# é necessário guardar o valor de cada eletrodo, ou seja, quantas vezes ele aparece nos arquivos
outputA = process_files(all_files, 'dataset/small/a_1_co2a0000364')
media(electrodeQtd, outputA)

#electrodeQtd = {}
#outputB = {}
#outputB = process_files(all_files, 'dataset/small/c_1_co2c0000337')
#media(electrodeQtd, outputB)

# Salvando os dados

Com todos os dados dos eletrodos, bastas salvar esse objeto em `.json`.

In [4]:
import json

def saveJson(output, name):
    # para o json é realmente muito fácil
    fileOutput = open(name, 'w')
    json.dump(output, fileOutput)
    fileOutput.close()

saveJson(outputA, 'dataset/small/a_1_co2a0000364.json')
#saveJson(outputB, 'dataset/small/c_1_co2c0000337.json')

def saveFile():
    # para csv é mais complicado, pois os dados tem que estar em colunas e não em linhas
    fileOutput = open('result_large_test.csv', 'w')

    # recuperando todos os eletrodos e ordenando-os
    electrodes = list(output.keys())
    electrodes.sort()

    # salvando o cabeçalho do arquivo
    virgula = ','
    for pos, electrode in enumerate(electrodes):
        # se for o último elemento, então adiciona o '\n' ao invés da ','
        if pos == len(electrodes)-1:
            virgula = '\n'

        fileOutput.write(electrode  + virgula)

    # agora falta salvar os valores
    for i in range(0, 255):
        for pos, electrode in enumerate(electrodes):
            if pos == len(electrodes)-1:
                virgula = '\n'
            else:
                virgula = ','

            fileOutput.write(output[electrode][i].__str__() + virgula)

    fileOutput.close()